In [1]:
import torch
import os
from torchvision import transforms, datasets,models
from torch.utils import data
from torch import optim
print(torch.__version__)

2.0.1+cu117


In [2]:
###########################
# Checking if GPU is used
###########################

use_cuda=torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
device=torch.device("cuda:0" if use_cuda else "mps" if use_mps else "cpu")
device

device(type='cuda', index=0)

In [3]:
!nvidia-smi

Fri Jul 12 00:27:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM3-32GB           On  | 00000000:34:00.0 Off |                    0 |
| N/A   33C    P0              75W / 350W |    737MiB / 32768MiB |     18%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
########################################
# Setting basic parameters for the model
########################################           
         
batch_size=16
max_epochs=10
lr=0.0001

NUM_CLIENTS = 10

NUM_DATA_SPLIT = NUM_CLIENTS

ROUNDS = 10

# added this just to done some experiments. Remove it to execute correct code.
# It should be same as NUM_CLIENTS
# NUM_DATA_SPLIT = 40

num_classes = 2


# name of new dataset
dataset = "Real_images_processed"

In [4]:
train_root_dir=os.path.join(dataset, 'train')   
val_root_dir= os.path.join(dataset, 'validation') 
test_root_dir= os.path.join(dataset, 'test') 
model_path='./fl_checkpointsswintransformer_e10_c10_r10'  # set path to the folder that will store model's checkpoints

n_classes=2  # number of classes used for training

global val_f1_max


try:
    if not os.path.exists(model_path):
        os.makedirs(model_path)
except OSError as err:
    print(err)
    
print('train root', train_root_dir)

print("Directory '% s' created" % model_path)
filename='results_e'+str(max_epochs)+'_'+'b'+str(batch_size)+'_'+'lr'+str(lr)+'_'+'cnn'   #filename used for saving epoch-wise training details and test results 

train root Real_images_processed/train
Directory './fl_checkpointsswintransformer_e10_c10_r10' created


## Federated Learning

In [5]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

print(
    f"Training on {device} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

2024-07-12 12:58:41.270212: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 12:58:45.220009: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Training on cuda:0 using PyTorch 2.0.1+cu117 and Flower 1.9.0


### Custom Data Loader

In [6]:
def load_datasets(num_clients: int):
    # ####################################
    # # Training
    # ####################################

    trans={
        # Train uses data augmentation
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            # transforms.RandomRotation(degrees=15),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.4762, 0.3054, 0.2368], [0.3345, 0.2407, 0.2164])
        ]),
        # Validation does not use augmentation
        'valid':
        transforms.Compose([
            transforms.Resize((224, 224)),            
            transforms.ToTensor(),
            transforms.Normalize([0.4762, 0.3054, 0.2368], [0.3345, 0.2407, 0.2164])
        ]),

        # Test does not use augmentation
        'test':
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.4762, 0.3054, 0.2368], [0.3345, 0.2407, 0.2164])
        ]),
    }

    training_dataset= datasets.ImageFolder(train_root_dir, transform=trans['train'])
    validation_dataset= datasets.ImageFolder(val_root_dir, transform=trans['valid'])
    test_dataset= datasets.ImageFolder(test_root_dir, transform=trans['test'])

    print('Number of Training set images:{}'.format(len(training_dataset)))
    print('Number of Validation set images:{}'.format(len(validation_dataset)))
    print('Number of Test set images:{}'.format(len(test_dataset)))

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(training_dataset) // num_clients
    lengths = [partition_size] * num_clients
    diff = len(training_dataset) - sum(lengths)
    lengths[-1] = lengths[-1] + diff
    train_datasets = random_split(training_dataset, lengths, torch.Generator().manual_seed(42))

    partition_size = len(validation_dataset) // num_clients
    lengths = [partition_size] * num_clients
    diff = len(validation_dataset) - sum(lengths)
    lengths[-1] = lengths[-1] + diff
    val_datasets = random_split(validation_dataset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for tr_ds in train_datasets:
        trainloaders.append(data.DataLoader(tr_ds, batch_size=batch_size, shuffle=True))
    for val_ds in val_datasets:
        valloaders.append(data.DataLoader(val_ds, batch_size=batch_size))

    testloader = data.DataLoader(test_dataset,batch_size)
    
    print("\nDatassets after split...")
    print('Number of Training set images:{}'.format(len(trainloaders)))
    print('Number of Validation set images:{}'.format(len(valloaders)))

    
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_DATA_SPLIT)

Number of Training set images:196
Number of Validation set images:55
Number of Test set images:30

Datassets after split...
Number of Training set images:10
Number of Validation set images:10


### Model Training

In [7]:
def train(model, trainloader, epochs: int):
    """Train the network on the training set."""
    optimizer=optim.Adam(model.parameters(), lr, weight_decay=1e-4)

    scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',patience=4,verbose=True)

    criterion = nn.CrossEntropyLoss()

    model.train()
    print("Starting client training...")
    for epoch in range(epochs):
        print(f"Running epoch {epoch}/{epochs}")
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item() * images.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")
        scheduler.step(epoch_loss)
#             epoch_loss += loss
#             total += labels.size(0)
#             correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
#         epoch_loss /= len(trainloader.dataset)
#         epoch_acc = correct / total
#         print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(model, testloader):
    """Evaluate the network on the entire test set."""
    criterion = nn.CrossEntropyLoss()

    correct, total, loss = 0, 0, 0.0
    model.eval()
    print("Starting client validation...")
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    print(f"Validataion accuracy = {accuracy}")
    return loss, accuracy

### Custom Strategy

In [8]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        
        net = create_pretrained_densenet(2)
        net.to(device)
        # get initial parameters for starting point.
        ndarrays = get_parameters(net)
        # ndarrays_to_parameters function converst numpy ndarray into io.bytes (tensors).
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        print("Starting configure fit function...")
        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        
        print(f"Sample size = {sample_size} Min Num Clients = {min_num_clients} Clients = {clients}")

        # Create custom configs
        n_clients = len(clients)
        half_clients = n_clients // 2
        standard_config = {"lr": 0.001}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        
        print(f"Number of clients = {n_clients}")
        for idx, client in enumerate(clients):
            if idx < half_clients:
                fit_configurations.append((client, FitIns(parameters, standard_config)))
            else:
                fit_configurations.append(
                    (client, FitIns(parameters, higher_lr_config))
                )
        print("Fit Configurations")        
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        
        return parameters_aggregated, metrics_aggregated



    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        loss = 1
        metric = {}    
        
        try:
            parameters_ndarrays = parameters_to_ndarrays(parameters)
            len(parameters_ndarrays)
            # Let's assume we won't perform the global model evaluation on the server side.
            print("Evaluating global model")                
            global_model = create_pretrained_densenet(2)
            global_model.to(device)

            set_parameters(global_model, parameters_ndarrays)
            print("Parameter set")
            loss, accuracy = test(global_model, testloader)
            print(f"Global evaluation round {server_round} loss = {float(loss)} accuracy = {float(accuracy)}")
            
            # Save your weights here.
            best_model_path = model_path+f'/fl_densenet_{server_round}.pth'
            torch.save({'round':server_round,
                'state_dict':global_model.state_dict(),
                'loss':loss,
                'acc':accuracy},
                best_model_path    
                )
            print(f"Model saved at {best_model_path}")
            
        except:
            print("Parameter type is not ndarray")
        
        return float(loss), metric

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [9]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, trainloader, valloader, max_epochs):
        self.cid = cid
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader
        self.max_epochs = max_epochs

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.model)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.model, parameters)
        print("Client parameters set")
        train(self.model, self.trainloader, epochs=self.max_epochs)
        print("Training completed!")
        return get_parameters(self.model), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.model, parameters)
        loss, accuracy = test(self.model, self.valloader)
        print(f"Evaluation loss = {float(loss)} accuracy = {float(accuracy)}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

    
import timm

def create_pretrained_densenet(num_classes):
    model = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=2)
    return model
# def create_pretrained_densenet(num_classes):
#     model = models.vit_b_16(pretrained=True)
#     # Replace the classifier with a new one (the number of classes in your dataset)
#     num_ftrs = model.classifier.in_features
#     model.classifier = nn.Linear(num_ftrs, num_classes)
#     return model



def client_fn(cid) -> FlowerClient:
    model = create_pretrained_densenet(num_classes)
    model.to(device)

    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    return FlowerClient(cid, model, trainloader, testloader, max_epochs)

### Creating Flower Client

In [10]:
def get_parameters(model) -> List[np.ndarray]:
    listOfArray = [val.cpu().numpy() for _, val in model.state_dict().items()]
    print(f"Length of params: {len(listOfArray)}")
    return listOfArray

def set_parameters(model, parameters: List[np.ndarray]):
    print(f"Updating parameters {len(parameters)}")
    
    ord_dict = {}
    # this is added for densenet parameters inconsistency.
    for k, v, p in zip(model.state_dict().keys(), model.state_dict().values(), parameters):
        try:
            len(p)
            ord_dict[k] = torch.Tensor(p)
        except:
#             print(f"Value of V = {v}")
            ord_dict[k] = torch.Tensor(v)
    
#     params_dict = zip(model.state_dict().keys(), parameters)
#     st_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})

    st_dict = OrderedDict(ord_dict)
    model.load_state_dict(st_dict, strict=True)
    print("Setting parameters...")

In [11]:
# Create an instance of the model and get the parameters
# params = get_parameters(create_pretrained_densenet(num_classes).to(device))
# print("ParamsIn: ", params)

# Pass parameters to the Strategy for server-side parameter initialization
# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=0.3,
#     fraction_evaluate=0.3,
#     min_fit_clients=1,
#     min_evaluate_clients=1,
#     min_available_clients=NUM_CLIENTS,
# #     initial_parameters=fl.common.ndarrays_to_parameters(params),
# )

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if use_cuda:
    client_resources = {'num_gpus': 1}

# Start simulation
# fl.simulation.start_simulation(
#     client_fn=client_fn,
#     num_clients=NUM_CLIENTS,
#     config=fl.server.ServerConfig(num_rounds=1),  # Just three rounds
#     strategy=strategy,
#     client_resources=client_resources,
# )

history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUNDS),  # Just three rounds
    strategy=FedCustom(),
    client_resources=client_resources,
)


print(f"End of Simulation History = {history}")

INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-07-12 12:58:55,935	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'accelerator_type:V100': 1.0, 'node:__internal_head__': 1.0, 'node:10.128.0.31': 1.0, 'memory': 1350849895424.0, 'CPU': 96.0, 'object_store_memory': 200000000000.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
/tmp/ipykernel_4135510/24281487.py:29: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_cpus=1` for each client.
  history = fl.simulation.start_simulation(
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
(pid=4144273) 2024-07-12 12:59:21.321108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find T

Length of params: 329


INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters


Evaluating global model
Updating parameters 329
Setting parameters...
Parameter set
Starting client validation...
Validataion accuracy = 0.4666666666666667
Global evaluation round 0 loss = 0.054304216305414835 accuracy = 0.4666666666666667


INFO :      initial parameters (loss, other metrics): 0.054304216305414835, {}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

(ClientAppActor pid=4144273) [Client 3] evaluate, config: {}
(ClientAppActor pid=4144273) Updating parameters 329
(ClientAppActor pid=4144273) Setting parameters...
(ClientAppActor pid=4144273) Starting client validation...
(ClientAppActor pid=4144273) Validataion accuracy = 0.8333333333333334
(ClientAppActor pid=4144273) Evaluation loss = 0.09184489250183106 accuracy = 0.8333333333333334


(ClientAppActor pid=4144273) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=4144273) [Client 8] evaluate, config: {}
(ClientAppActor pid=4144273) Updating parameters 329
(ClientAppActor pid=4144273) Setting parameters...
(ClientAppActor pid=4144273) Starting client validation...


INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 7466.76s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.04820530116558075
INFO :      		round 2: 0.04684884684781233
INFO :      		round 3: 0.060025711357593534
INFO :      		round 4: 0.0669787441690763
INFO :      		round 5: 0.06496138274669648
INFO :      		round 6: 0.08114009896914164
INFO :      		round 7: 0.07091672817866006
INFO :      		round 8: 0.07146279215812681
INFO :      		round 9: 0.0941953976949056
INFO :      		round 10: 0.09184489250183105
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.054304216305414835
INFO :      		round 1: 0.04820530116558075
INFO :      		round 2: 0.046848846847812334
INFO :      		round 3: 0.060025711357593534
INFO :      		round 4: 0.06697874416907629
INFO :      		round 5: 0.06496138274669647
INFO :      		round 6: 0.08114009896914164
INFO :      		round 7: 0.070916

(ClientAppActor pid=4144273) Validataion accuracy = 0.8333333333333334
(ClientAppActor pid=4144273) Evaluation loss = 0.09184489250183106 accuracy = 0.8333333333333334
End of Simulation History = History (loss, distributed):
	round 1: 0.04820530116558075
	round 2: 0.04684884684781233
	round 3: 0.060025711357593534
	round 4: 0.0669787441690763
	round 5: 0.06496138274669648
	round 6: 0.08114009896914164
	round 7: 0.07091672817866006
	round 8: 0.07146279215812681
	round 9: 0.0941953976949056
	round 10: 0.09184489250183105
History (loss, centralized):
	round 0: 0.054304216305414835
	round 1: 0.04820530116558075
	round 2: 0.046848846847812334
	round 3: 0.060025711357593534
	round 4: 0.06697874416907629
	round 5: 0.06496138274669647
	round 6: 0.08114009896914164
	round 7: 0.07091672817866007
	round 8: 0.07146279215812683
	round 9: 0.0941953976949056
	round 10: 0.09184489250183106



## Inference

In [ ]:
# import torch
# import torchvision.transforms as transforms
# from PIL import Image
# import os
# from torchvision import datasets, models
# from torch.utils.data import DataLoader

In [40]:
# # Define device
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda:0" if use_cuda else "cpu")

In [17]:
# # Load model
# num_classes = 2
# model = create_pretrained_densenet(num_classes)
# model.to(device)

In [18]:
# # Load best model weights
# best_model_path = "./fl_checkpointsdensenet121/fl_densenet_15.pth"
# checkpoint = torch.load(best_model_path)
# model.load_state_dict(checkpoint['state_dict'])
# print("Model loaded successfully")

In [43]:
# # Define transform
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.4762, 0.3054, 0.2368], std=[0.3345, 0.2407, 0.2164])
# ])

# # Define test dataset and dataloader
# test_root_dir = os.path.join('./../../processed_RealDS', 'test')
# test_dataset = datasets.ImageFolder(test_root_dir, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [3]:
# # Load your single image
# PATH_DATASET = './../../processed_RealDS'

# image_path = os.path.join(PATH_DATASET, 'test','Disease','Image_114.jpeg')  # Replace 'path_to_your_image.jpg' with the path to your image
# image = Image.open(image_path)


In [34]:
# image = Image.open(image_path)

# # Apply the transform to the image
# image = transform(image)

# # Add batch dimension
# image = image.unsqueeze(0)
# image = image.to(device)


In [19]:
# # Perform inference and calculate accuracy
# model.eval()
# correct = 0
# total = 0
# class_labels = ['Healthy', 'Disease']  # Replace with your actual class labels


In [20]:
# # Perform inference
# with torch.no_grad():
#     output = model(image)

#     # Get the predicted class index
#     _, predicted = torch.max(output, 1)
#     predicted_class_index = predicted.item()
    
#     # Interpret the results (assuming you have a class label mapping)
#     class_labels = ['Healthy','Disease' ]  # Replace with your actual class labels
#     predicted_class_label = class_labels[predicted_class_index]
#     print('Predicted class:', predicted_class_label)

In [21]:
# print('Predicted class:', predicted_class_label)

## all predictions

In [22]:
# with torch.no_grad():
#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         print(f"Predicted class: {class_labels[predicted.item()]}")

# accuracy = correct / total
# print(f'Overall test accuracy: {accuracy * 100:.2f}%')

In [12]:
# tmp_model = create_pretrained_densenet(num_classes).to(device)

# params = get_parameters(tmp_model)

# # it will return byteio(numpyArray(param))
# array_to_param = fl.common.ndarrays_to_parameters(params)

# param_to_arr = fl.common.parameters_to_ndarrays(array_to_param)

# print(type(param_to_arr))

# for i, v in enumerate(param_to_arr):
#     try:
#         print(len(v))
#     except:
#         print(v)
#         param_to_arr[i] = np.array([0])
# set_parameters(tmp_model, param_to_arr)

# Testing

In [15]:

############################
#         Test
############################
test_list=[]
test_model_path = "fl_checkpointsdensenet121_e5_c5_r5/fl_densenet_5.pth"
print('Best model path:{}'.format(test_model_path))
best_model=models.densenet121(weights=False).to(device)

n_inputs = best_model.classifier.in_features
best_model.classifier = nn.Sequential(
              nn.Linear(n_inputs, n_classes),               
              nn.LogSoftmax(dim=1))


checkpoint=torch.load(test_model_path,map_location=device)   # loading best model
best_model.load_state_dict(checkpoint['state_dict'])
best_model.to(device)
best_model.eval()

Best model path:fl_checkpointsdensenet121_e5_c5_r5/fl_densenet_5.pth


/home/sarangs/D1/chameleon_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [16]:
from torch.utils.data import DataLoader

# Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4762, 0.3054, 0.2368], std=[0.3345, 0.2407, 0.2164])
])

# Define test dataset and dataloader
test_root_dir = os.path.join('Real_images_processed', 'test')
test_dataset = datasets.ImageFolder(test_root_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [17]:
# Org test images: 111.jpg, 119.jpg
# Other org: 662.jpg, 841.jpg, 731.jpg
correct, total, loss = 0, 0, 0.0
count = 1
class_labels = ['Disease', 'Healthy']
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f"{count} Predicted class: {class_labels[predicted.item()]} | Actual Class: {class_labels[labels]}")
        count += 1
accuracy = correct / total
print(f'Overall test accuracy: {accuracy * 100:.2f}%')

1 Predicted class: Disease | Actual Class: Disease
2 Predicted class: Disease | Actual Class: Disease
3 Predicted class: Disease | Actual Class: Disease
4 Predicted class: Disease | Actual Class: Disease
5 Predicted class: Healthy | Actual Class: Disease
6 Predicted class: Disease | Actual Class: Disease
7 Predicted class: Healthy | Actual Class: Disease
8 Predicted class: Disease | Actual Class: Disease
9 Predicted class: Disease | Actual Class: Disease
10 Predicted class: Disease | Actual Class: Disease
11 Predicted class: Disease | Actual Class: Disease
12 Predicted class: Disease | Actual Class: Disease
13 Predicted class: Disease | Actual Class: Disease
14 Predicted class: Disease | Actual Class: Disease
15 Predicted class: Healthy | Actual Class: Healthy
16 Predicted class: Healthy | Actual Class: Healthy
17 Predicted class: Healthy | Actual Class: Healthy
18 Predicted class: Healthy | Actual Class: Healthy
19 Predicted class: Healthy | Actual Class: Healthy
20 Predicted class: D